In [25]:
import os
print(multiprocessing.cpu_count())

print(len(os.sched_getaffinity(0)))

8
8


In [11]:
from matching.games import StableMarriage

def parse_preferences(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()

    men_prefs = {}
    women_prefs = {}
    for line in lines:
        parts = line.split(':')
        entity, prefs = parts[0].strip(), parts[1].strip().split()
        if entity.startswith('m'):
            men_prefs[entity[1:]] = prefs
        else:
            women_prefs[entity[1:]] = prefs
            
    return men_prefs, women_prefs

men_prefs, women_prefs = parse_preferences('./data.txt')
men_prefs1, women_prefs1 = parse_preferences('./dataSMI.txt')

#print(men_prefs)



In [12]:
import random
def create_chromosome_repeat(men_preferences, women_preferences):
    men = list(men_preferences.keys())
    women = list(women_preferences.keys())
    matches = []
    chosen_women = set()

    for man in men:
        available_women = [w for w in men_preferences[man] if w not in chosen_women]
        if available_women:
            woman = random.choice(available_women)
            matches.append((man, woman))
            chosen_women.add(woman)
            
    return matches


def create_chromosome(men_preferences, women_preferences):
    chromosome = create_chromosome_repeat(men_preferences, women_preferences)
    stable, _ = is_stable(chromosome, men_preferences, women_preferences)
    while not stable:
        chromosome = create_chromosome_repeat(men_preferences, women_preferences)
        stable, _ = is_stable(chromosome, men_preferences, women_preferences)
    return chromosome

In [13]:
def is_stable(chromosome, men_preferences, women_preferences):
    def prefers(new, current, preferences):
        # Return True if the individual prefers the new partner over the current partner
        return preferences.index(new) < preferences.index(current)
    
    # Extract the current matches from the chromosome
    man_to_woman = {man: woman for man, woman in chromosome}
    woman_to_man = {woman: man for man, woman in chromosome}
    
    blocking_pairs = 0
    
    # Check each man and woman to see if they form a blocking pair
    for man, woman in chromosome:
        man_prefs = men_preferences[man]
        woman_prefs = women_preferences[woman]
        
        # Check if there is any woman in man's preference list that he prefers more than his current match
        for preferred_woman in man_prefs:
            if preferred_woman == woman:
                break  # He prefers his current match the most, no blocking pair possible beyond this point
            
            preferred_woman_current_man = woman_to_man[preferred_woman]
            # Check if the preferred woman prefers this man over her current match
            if prefers(man, preferred_woman_current_man, women_preferences[preferred_woman]):
                blocking_pairs += 1
                break
        
        # Check if there is any man in woman's preference list that she prefers more than her current match
        for preferred_man in woman_prefs:
            if preferred_man == man:
                break  # She prefers her current match the most, no blocking pair possible beyond this point

            if preferred_man in man_to_woman:
                preferred_man_current_woman = man_to_woman[preferred_man]
                # Check if the preferred man prefers this woman over his current match
                if prefers(woman, preferred_man_current_woman, men_preferences[preferred_man]):
                    blocking_pairs += 1
                    break
    
    # The chromosome is stable if there are no blocking pairs
    is_stable = blocking_pairs == 0
    
    return is_stable, blocking_pairs


In [14]:
def calculate_happiness_cost(engagements, men_prefs, women_prefs):
    return sum(men_prefs[man].index(woman) + women_prefs[woman].index(man)
               for man, woman in engagements)

def calculate_egalitarian_cost(engagements, men_prefs, women_prefs):
    return sum(abs(men_prefs[man].index(woman) - women_prefs[woman].index(man))
               for man, woman in engagements)

def calculate_stability(engagements, men_prefs, women_prefs):
    is_chromosome_stable, blocking_pairs_count = is_stable(engagements, men_prefs, women_prefs) 
    return blocking_pairs_count

def calculate_fitness(engagements, men_prefs, women_prefs):
    happiness_cost = calculate_happiness_cost(engagements, men_prefs, women_prefs)
    egalitarian_cost = calculate_egalitarian_cost(engagements, men_prefs, women_prefs)
    stability = calculate_stability(engagements, men_prefs, women_prefs)
    
    # Combine the costs into a fitness score (you may want to weigh these differently)
    fitness = stability, egalitarian_cost + happiness_cost
    return fitness

In [15]:
def one_point_crossover(parent1, parent2):
    # Convert the engagements to lists for easy crossover
    men1, women1 = zip(*parent1.items())
    men2, women2 = zip(*parent2.items())

    crossover_point = random.randint(1, len(men1) - 2)
    new_men = men1[:crossover_point] + men2[crossover_point:]
    new_women = women1[:crossover_point] + women2[crossover_point:]

    # Remove duplicates to maintain valid engagements
    engagements = {}
    for m, w in zip(new_men, new_women):
        if w not in engagements.values():
            engagements[m] = w
            
    return engagements


In [16]:
def swap_mutation(individual):
    men = list(individual.keys())
    m1, m2 = random.sample(men, 2)
    individual[m1], individual[m2] = individual[m2], individual[m1]
    return individual


In [21]:
import multiprocessing
from multiprocessing import Pool
# Modified evolutionary algorithm with parallelization
def evolutionary_algorithm(men_prefs, women_prefs, pop_size, generations):
    
    with Pool(multiprocessing.cpu_count()) as pool:
        population = pool.map(create_chromosome(men_prefs, women_prefs), range(pop_size))
    
    best_fit = None
    best_solution = None

    for _ in range(generations):
        fitness_scores = [(calculate_fitness(chromosome, men_prefs, women_prefs), chromosome) for chromosome in population]
        fitness_scores = sorted(fitness_scores, key=lambda x: x[0])
        
        parents = list(map(lambda x: x[1], fitness_scores))
        fitness_scores = list(map(lambda x: x[0], fitness_scores))

        new_population = parents[:len(parents) // 2]
        
        with Pool(multiprocessing.cpu_count()) as pool:
            new_chromosomes = pool.map(create_chromosome(men_prefs, women_prefs), range(pop_size // 2))
        
        new_population += new_chromosomes
        assert(len(new_population) == pop_size)
        
        # Check if we have a new best solution
        max_fit_index = fitness_scores.index(max(fitness_scores))
        if best_fit is None or fitness_scores[max_fit_index] > best_fit:
            best_fit = fitness_scores[max_fit_index]
            best_solution = new_population[max_fit_index]

        population = new_population

    return best_solution, best_fit

In [26]:
# Now, we can call the evolutionary algorithm with the correct path to your data file
if __name__ == "__main__":
    best_solution, best_fit = evolutionary_algorithm(men_prefs, women_prefs, pop_size=100, generations=10)
    print("Best Solution:")
    for man, woman in best_solution:
        print(f"{man} is matched with {woman}")
    print(f"Best Fitness: {best_fit}")


KeyboardInterrupt: 